# Titanic - Preprocessing Data


## Introduction

> The sinking of the RMS Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

> One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

> In this contest, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

> This Kaggle Getting Started Competition provides an ideal starting place for people who may not have a lot of experience in data science and machine learning."

From the competition homepage: https://www.kaggle.com/competitions/titanic

#### The ultimate goal: 
- Detecting all of the error values (missing values, outlier values)

## Preparation

### Import necessary libraries

In [1]:
# turn off warning
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load train.csv file

In [2]:
# titanic = pd.read_csv('/kaggle/input/titanic/train.csv')
titanic = pd.read_csv('../../../../input/train.csv')
df = titanic.copy()
test_dataset = pd.read_csv('../../../../input/test.csv')
df_test = test_dataset.copy()
print("Successfully load training data.")

Successfully load training data.


## Have a look at the dataset

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

We can see that `Age`, `Cabin` and `Embarked` in train dataset and `Age`, `Cabin` and `Fare` in test dataset have missing values, we will explore number of missing value and it's proportion.

Look at the features information, there are four categorical variables, we will convert it into `category` type to implement analysis better.

In [5]:
for x in ['Pclass', 'Sex', 'Embarked']:
    df[x] = df[x].astype('category')
    df_test[x] = df_test[x].astype('category')
df['Survived'] = df['Survived'].astype('category')

In [6]:
print("===== Number of missing values (train dataset) =====\n")
print(df[['Age', 'Cabin', 'Embarked']].isnull().sum())
print('\n')
print("===== Number of missing values (test dataset) =====\n")
print(df_test[['Age', 'Cabin', 'Fare']].isnull().sum())

===== Number of missing values (train dataset) =====

Age         177
Cabin       687
Embarked      2
dtype: int64


===== Number of missing values (test dataset) =====

Age       86
Cabin    327
Fare       1
dtype: int64


In [7]:
print("===== Percentage of missing values on train dataset =====\n")
print(df[['Age', 'Cabin', 'Embarked']].isnull().sum() / len(df) * 100)
print('\n')
print("===== Percentage of missing values on test dataset =====\n")
print(df_test[['Age', 'Cabin', 'Fare']].isnull().sum() / len(df_test) * 100)

===== Percentage of missing values on train dataset =====

Age         19.865320
Cabin       77.104377
Embarked     0.224467
dtype: float64


===== Percentage of missing values on test dataset =====

Age      20.574163
Cabin    78.229665
Fare      0.239234
dtype: float64


The `Cabin` has lots of missing data, so we might need to find a way to handle it later.

We should also check if any rows inside the dataset duplicate each other in order to deal with it as these could affect negatively on training length.

In [8]:
print(df[df.duplicated()])

Empty DataFrame
Columns: [PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked]
Index: []


There are no duplicate rows in the dataset, which is good.

We want to check it's uniqueness of each column.

In [9]:
df.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

From the result, we could see that some features do have lots of unique values, but some features (`Survived`, `Pclass`, `Sex`, `Embarked`) only have 2 - 3 different values, as it is categorical (we don't count `SibSp`, `Parch` here because it makes sense).

## Handle missing value

We need to find solutions to handle missing value.
From the output detecting missing value, here's the proportion of missing value for `Age`, `Cabin`, and `Embarked` in train dataset:

- Age: 19.87%
- Cabin: 77.10%
- Embarked: 0.22%

For test dataset, here's the proportion of missing value for `Age`, `Cabin`, and `Fare`:
- Age: 20.57%
- Cabin: 78.23%
- Fare: 0.24%

For `Cabin` features, although the missing value percentage are pretty high, it could be a good information to show passenger's room level, which is a really crucial detail related to survivability. Furthermore, we can base on that information to know if the passengers stayed at the front part (bow) of the Titanic, which is the part of the ship that sank first.

For `Age` features, it could related to the survivability, as children and elderly usually be prioritized.

For `Embarked` features, it could related to passenger's room level (passengers go on the ship first might have room that are in higher level).

Because of that, I think that we should not remove these features. We need to handle value instead.

For testing dataset, we also have missing value in `Fare` feature. We will handle it.

### Handle missing age value

Honorific title inside passengers' name could tell us a little about their age.

At first, we splited value `Age` into 5 main groups: Mr, Mrs, Master, Miss, and other in order to not only impute missing values but also capture potential age-related patterns.

But after looking at survivability rate of each group, we decided not to complicate things and just impute missing value with median age of the whole dataset. Except for `Master` group, we will impute missing value with median age of this group as title "Master" is only for people under 12.

In [10]:
master = df['Name'].str.contains(r',\s*Master.', regex=True)
df_master = df[master].copy()

mean_age = df['Age'].mean()
mean_age_master = df_master['Age'].mean()

df_master['Age'] = df_master['Age'].fillna(mean_age_master)
df[master] = df_master

df['Age'] = df['Age'].fillna(mean_age)
df_test['Age'] = df_test['Age'].fillna(mean_age)

### Handle missing values from Cabin, Embarked (train dataset) and Fare (test dataset)

Perhaps the only features that we could use to guess people's cabin is Ticket number, but since there are no ticket number system, we could not conclude anything from that.

At first, I thought it would be possible to fill missing `Cabin` value by looking at passengers' ticket class, fare, and embarked point, but after some research, I found out that there is no clear relationship between these features and `Cabin` value.

Therefore, I will just remove the `Cabin` feature, as it has too many missing values and we could not find a way to fill them.

As for Embarked value, I will choose "S" to fill that, as the primary embarked point was Southampton (Titanic trip started from there)

As for Fare value in testing dataset, I will fill it with median of Fare value.

In [11]:
df = df.drop(['Cabin'], axis=1)
df_test = df_test.drop(['Cabin'], axis=1)

df['Embarked'] = df['Embarked'].fillna("S")
df_test['Embarked'] = df_test['Embarked'].fillna("S")

fare_mean = df['Fare'].mean()
df_test['Fare'] = df_test['Fare'].fillna(fare_mean)

In [12]:
print("===== Percentage of missing values =====")
df[['Age', 'Embarked']].isnull().sum() / len(df) * 100

===== Percentage of missing values =====


Age         0.0
Embarked    0.0
dtype: float64

## Detecting outliers

This section will mainly focus on addressing outlier values in numeric features.

Based on the outputs from the boxplot from EDA Analysis, we could see that `Age`, `Fare`, `SibSp`, and `Parch` features have outlier values. However, since `SibSp` and `Parch` features are not continuous (both features have a limited number of unique values), we will not handle outlier values in these two features, as it could affect negatively on the model.

To handle outlier values, IQR method is used to find the lower bound and upper bound, then replace outlier values with values within these bounds using techniques such as capping or imputation.

In [13]:
num_cols = ['Age', 'Fare']
def detect_outliers_iqr(df, col, factor=1.5):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = max(Q1 - factor * IQR, 0)  # Age cannot be negative
    upper_bound = Q3 + factor * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    # print(f'{col} - Outliers (IQR): {len(outliers)}, Lower: {lower_bound:.2f}, Upper: {upper_bound:.2f}')
    return outliers, lower_bound, upper_bound


for col in num_cols:
    outliers, lower_bound, upper_bound = detect_outliers_iqr(df, col)
    print(f'Total outliers in {col}: {len(outliers)}')

Total outliers in Age: 66
Total outliers in Fare: 116


Base on the outcome, we could see that:
- Total outliers in Age: 66
- Total outliers in Fare: 116

The number of outliers in `Fare` is quite high, we will have to handle it carefully in the next part.

## Save file

We will save train and test files to use for the future

In [14]:
df.to_csv('./data/exp1_train.csv', index=False)
df_test.to_csv('./data/exp1_test.csv', index=False)
print("Successfully saved preprocessed data.")

Successfully saved preprocessed data.


# The end